In [ ]:
# 주피터 서버에 패키지 설치용
# %pip install dotenv


In [ ]:
# OpenAI 사용 예시
#from openai import OpenAI
#client = OpenAI()
#
#response = client.responses.create(
#    model="gpt-5",
#    tools=[{"type": "web_search"}],
#    input="What's the current weather in Seoul, South Korea? Please provide temperature, conditions, and today's forecast."
#)
#
# print(response.id) # resp_68b7e1e7b7c08194ad782e3fcb7e5d3a06624d1770548f9c


In [129]:
# 필요한 패키지 설치
from openai import OpenAI
from pathlib import Path

import os
from dotenv import load_dotenv


In [130]:
# 로컬에 있는 3장 pdf 파일과 목차 경로 정의
chat3_index_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\목차.txt")
actions_level_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\위기경보수준별조치사항.pdf")
docs_path = [chat3_index_docs_path, actions_level_docs_path]

In [127]:
# openai api key & 환경변수 설정

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o-mini"
PROMPT = """
            당신은 재난 대응 전문가이자 온톨로지 엔지니어입니다.

            ## 주요 역할
            - PDF 문서와 목차.txt를 정확하게 분석하여 계층적 클래스 구조 추출
            - OWL/RDF TTL 표준에 맞는 정확한 온톨로지 생성
            - 문서의 모든 세부사항을 누락 없이 반영하여 완벽한 온톨로지 구축

            ## 문서 분석 기준
            - **문서명**: [자연재난-1] (LH)풍수해 현장조치 행동매뉴얼 제정(안)_ver1.0_2024_03-part-4_위기경보 수준별 조치사항
            - **목표**: 태풍·호우·대설 등 재난유형별 위기경보 수준별 조치사항을 표준 온톨로지로 모델링
            - **정확도 요구사항**: 문서 내용과 100% 일치, 추측 금지, 모든 세부사항 포함

            ## 클래스 추출 규칙

            ### 1. **최상위 클래스**
            - `ex:위기경보수준별조치사항`을 rdfs:Class로 선언

            ### 2. **계층 구조**
            - 목차의 모든 레벨(1, 2, 가, 나, 다, 라, 마, 사, 1), 2), 3))를 rdfs:subClassOf로 표현
            - 계층 깊이에 관계없이 모든 항목을 클래스로 변환

            ### 3. **네이밍 규칙**
            - 형식: `ex:위기경보수준별조치사항_1_관심_나_2_1_조치목록`
            - 목차 구조를 그대로 반영하여 일관성 유지

            ### 4. **페이지 정보**
            - PDF 하단 페이지 번호를 정확히 추출하여 `ex:페이지정보`로 표시
            - 모든 클래스에 페이지 정보 필수 포함

            ## 온톨로지 생성 프로세스

            ### **1단계: 초기 온톨로지 생성**
            - 목차를 기반으로 모든 클래스와 서브클래스 생성
            - 계층 구조를 정확히 반영
            - 페이지 정보 포함

            ### **2단계: 온톨로지 보완 (필요시)**
            - 기존 클래스 구조는 절대 변경하지 않음
            - 새로운 개념만 적절한 상위 클래스 아래에 서브클래스로 추가
            - 문서 내용과 100% 일치하는지 검증

            ### **3단계: 품질 검증**
            - 목차의 모든 항목이 클래스로 변환되었는지 확인
            - 계층 구조가 문서와 정확히 일치하는지 검증
            - 페이지 정보가 정확한지 확인
            - 네이밍 규칙이 일관되게 적용되었는지 확인

            ## 출력 형식

            ### **TTL 파일 구조**
            ```turtle
            @prefix ex:     <http://example.org/disaster#> .
            @prefix rdfs:   <http://www.w3.org/2000/01/rdf-schema#> .
            @prefix dcterms:<http://purl.org/dc/terms/> .
            @prefix owl:     <http://www.w3.org/2002/07/owl#> .
            @prefix xsd:     <http://www.w3.org/2001/XMLSchema#> .

            # 클래스 정의
            ex:위기경보수준별조치사항
                a              rdfs:Class ;
                rdfs:label     "위기경보 수준별 조치사항"@ko ;
                ex:페이지정보   "1" .

            ex:위기경보수준별조치사항_1_관심
                a              rdfs:Class ;
                rdfs:subClassOf ex:위기경보수준별조치사항 ;
                rdfs:label     "관심"@ko ;
                ex:페이지정보   "2" .

            # 하위 클래스들 계속...
            ```

            ### **출력 요구사항**
            - TTL 파일 전체를 한 번에 출력
            - 모든 클래스와 서브클래스 포함
            - 페이지 정보 정확히 표시
            - 한국어 라벨 사용
            - 계층 구조 명확히 표현

            ## 품질 검증 체크리스트

            ### **완성도 검증**
            □ 목차의 모든 항목이 클래스로 변환되었는가?
            □ 계층 구조가 문서와 정확히 일치하는가?
            □ 모든 클래스에 페이지 정보가 포함되었는가?

            ### **정확성 검증**
            □ 네이밍 규칙이 일관되게 적용되었는가?
            □ rdfs:subClassOf 관계가 올바르게 설정되었는가?
            □ 문서 내용과 100% 일치하는가?

            ### **표준 준수 검증**
            □ OWL/RDF TTL 표준 문법을 준수하는가?
            □ Prefix 선언이 올바른가?
            □ 모든 클래스가 rdfs:Class로 선언되었는가?

            ## 금지사항
            - 문서에 없는 내용을 추측하여 추가하지 말 것
            - 기존 클래스 구조를 임의로 변경하지 말 것
            - 페이지 정보를 생략하지 말 것
            - 계층 구조를 단순화하지 말 것

            ## 최종 목표
            문서의 모든 내용을 누락 없이, 정확하게, 계층적으로 구조화된 온톨로지로 변환하여 RAG 기반 대화형 재난 대응 시스템의 완벽한 지식베이스를 구축하세요.

            문서 내용을 정확하게 분석하여 완벽한 온톨로지를 생성하세요. 모든 세부사항을 포함하고, 정확성과 완성도를 최우선으로 하여 작업하세요.
    """

client = OpenAI(api_key=OPENAI_API_KEY)


In [128]:
from typing import List

In [131]:
def upload_files(paths:List[str]) -> List[str]:
    """
    Files Upload Function
    """
    file_ids = []

    for path in paths:
        with open(path, 'rb') as f:
            response = client.files.create(
                file=f, purpose="assistants"
                )
            file_ids.append(response.id)
    
    return file_ids

In [132]:
upload_files(docs_path)

['file-AkYB14GUSEi9EixdZaov5V', 'file-PY6GsaCiBWmZFzJRz83rgj']

In [ ]:
# beta 사용
def create_assistant_beta(file_ids:List[str]) -> str:
    """
    Assistant Create Function
    """
    try: 
        assistant = client.beta.assistants.create(
            instructions= PROMPT,
            name="Ontology Class & Subclass create Assistant",
            tools=[{"type": "file_search"}],
            model=MODEL,
        )

        print(f"Successfully created assistant: {assistant.id}")

        for file_id in file_ids:
            try:
                client.beta.assistants.files.create(
                    assistant_id=assistant.id,
                    file_id=file_id,
                )
                print(f"Successfully added file: {file_id} to assistant: {assistant.id}")
            except Exception as e:
                print(f"Error adding file: {file_id} to assistant: {assistant.id}")
        print("--End of Assistant Creation--")
        return assistant.id

    except Exception as e:
        print(f"Error creating assistant: {e}")
        return None